# OSNAP mooring array

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from paths import file_ex_ocn_ctrl, path_results
from xr_DataArrays import xr_DZ

In [ ]:
ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)

In [ ]:
NLAT = ds.TLAT.copy()
X, Y = np.meshgrid(np.arange(3600), np.arange(2400))
NLAT.values = Y

In [ ]:
DZT = xr_DZ('ocn')

In [ ]:
fig = plt.figure(figsize=(12,4))
ax = plt.axes((.05,.15,.9,.83),projection=ccrs.PlateCarree())
ax.set_extent([-65, -5, 50, 62], crs=ccrs.PlateCarree())

cax, kw = mpl.colorbar.make_axes(ax,location='bottom',pad=0.1,shrink=0.8)

im1 = ax.contour(ds.TLONG[:,200:1050],
                ds.TLAT[:,200:1050],
                NLAT[:,200:1050].where(ds.REGION_MASK[:,200:1050]>0),
                np.linspace(1700,1960,11),
                linewidths=1,
                colors='r',
                vmin=1700, vmax=1900,
                transform=ccrs.PlateCarree() )

im2 = ax.pcolormesh(ds.TLONG,
                    ds.TLAT,
                    DZT.sum(dim='z_t').where(ds.REGION_MASK>0),
                    cmap=cmocean.cm.deep,
                    vmin=0, vmax=4500,
                    transform=ccrs.PlateCarree() )

ax.coastlines(resolution='50m')
ax.gridlines(draw_labels=True)

cbar = fig.colorbar(im2, cax=cax, extend='max', **kw)
cbar.ax.tick_params(labelsize=14)
label = cbar.set_label('depth [m]', size=16)

plt.savefig(f'{path_results}/OSNAP/OSNAP_domain')